In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mv drive/MyDrive/py_files/ /content

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import py_files
from py_files import load_data, const, bird_info, models
import warnings
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import  math
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [ ]:
f_drive = "drive/MyDrive/"
# f_drive=""

In [ ]:
image_size = (256, 256)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f_drive+"filtered_birds/",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size
)


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f_drive+"filtered_birds/",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size
)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    f_drive+"val_ten_uk_birds/",
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size
)

In [ ]:
def normalise_data(data):
    normalization_layer = keras.layers.Rescaling(1./255)
    normalised_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    return normalised_train_ds

In [ ]:
def convert_tensor_to_flat_list(dataset):
    tmp_images = []
    tmp_labels = []
    for t_images, t_labels in dataset.as_numpy_iterator():
        tmp_images.append(t_images)
        tmp_labels.append(t_labels)
    
    images = []
    labels = []
    for batch in tmp_images:
        for image in batch:
            images.append(image)
        
    for batch in tmp_labels:
        for label in batch:
            labels.append(label)
    
    return images, labels

In [ ]:
train_images_ls, train_labels_ls = convert_tensor_to_flat_list(normalise_data(train_ds))


In [ ]:
bird_info.visualise_10_10_images(train_images_ls, train_labels_ls, np.arange(0,10))

Models

In [ ]:
INPUT_SHAPE=(256,256,3)
m= models.resNet_50_transfer_learning_model(INPUT_SHAPE)

In [ ]:
m.summary()

In [ ]:
m.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
history=m.fit(train_ds, validation_data=val_ds, epochs=30, verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)

In [ ]:
test_images, test_labels = convert_tensor_to_flat_list(val_ds)

In [ ]:
t_i = np.array(test_images)
t_l = np.array(test_labels)

In [ ]:
m.evaluate(val_ds)

In [ ]:
preds = m.predict(t_i)

In [ ]:
max_val_per_row=np.amax(preds, axis=1)
len(max_val_per_row)

In [ ]:
max_where=[]
for index, max_row in enumerate(max_val_per_row):
  a = np.where(preds[index]==max_row)
  max_where.append(int(a[0]))

In [ ]:

lab=np.ones((10,10))*np.arange(0,10)

In [ ]:
from sklearn.metrics import confusion_matrix 
import seaborn as sns
results = confusion_matrix(t_l, max_where)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(results, annot=True, fmt='d')
